# Telemetry Collection Agent (TCA) loads via COA

In [41]:
%disconnect Transcend

ERROR: Profile Transcend not connected

In [42]:
%connect TranscendDev

Connected: 'TranscendDev' connection activated for user 'sh186014'


In [ ]:
SET TIME ZONE 'America Pacific';

In [ ]:
-- Time right now:
Select Current_Timestamp(0);

<br><br><br>
# Build supporting datalab objects 
- Stage tables  (global temp)
- Target Tables (for demo purposes only, will eventually be TCA)
- Stored Procs  (move data from stage to target, for demo purposes only)

<br><br><br>
## DBC.DBCINFO Structures

In [ ]:
-- SQL for generating .csv file:
select 
  '{siteid}' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a

### APP_TCA_TMP.stg_tca_dbcinfo

In [ ]:
drop table APP_TCA_TMP.stg_tca_dbcinfo

In [ ]:
/* GLOBAL TEMP TABLE for stage data coming from COA
   note: all users of ADLSTE_COA datalab will need 
         SELECT, INSERT, DELETE, and UPDATE to this object */
CREATE SET GLOBAL TEMPORARY TABLE APP_TCA_TMP.stg_tca_dbcinfo 
( Site_ID       VARCHAR(128)
, InfoKey       VARCHAR(128)
, InfoData      VARCHAR(128)
) PRIMARY INDEX (Site_ID)
on commit preserve rows

In [ ]:
select * from APP_TCA_TMP.stg_tca_dbcinfo

## APP_TCA_TBL.coat_tca_dbcinfo

In [ ]:
drop view APP_TCA_TBL.coa_tca_dbcinfo

In [ ]:
-- TARGET TABLE:
CREATE TABLE APP_TCA_TBL.coa_tca_dbcinfo
( Site_ID        VARCHAR(128)
, InfoKey        VARCHAR(128)
, InfoData       VARCHAR(128)
, di_created_ts  TIMESTAMP(6)   DEFAULT CURRENT_TIMESTAMP(6)
, di_upd_ts      TIMESTAMP(6)   
, di_data_src_cd VARCHAR(4)
, di_proc_name   CHAR(30)
, di_batch_nbr   INTEGER
) PRIMARY INDEX(Site_ID);

## APP_TCA_VW.coa_tca_dbcinfo

In [ ]:
-- ABSTRACTION LAYER VIEW  (** OPTIONAL **) --
replace view APP_TCA_VW.coa_tca_dbcinfo as locking row for access
Select * from APP_TCA_TBL.coa_tca_dbcinfo;

## APP_TCA_TMP.sp_tca_dbcinfo

In [ ]:
drop procedure APP_TCA_TBL.sp_tca_dbcinfo

In [ ]:
REPLACE PROCEDURE APP_TCA_TBL.sp_tca_dbcinfo () 
SQL SECURITY CREATOR
BEGIN     /* this is a simplified example, excluding any audit logging */ 

    DECLARE proc_name CHAR(30);  
    DECLARE batch_nbr INTEGER ;  
    
    /* someone will need to fill these values from ETL */
    SET proc_name = 'Unknown';
    SET batch_nbr = 0        ;   
   
    
    MERGE INTO APP_TCA_TBL.coat_tca_dbcinfo  tgt
         USING APP_TCA_TMP.stg_tca_dbcinfo   stg
            ON (tgt.Site_ID = stg.Site_ID and
                tgt.InfoKey = stg.InfoKey)
    WHEN MATCHED THEN 
        UPDATE SET 
              InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_upd_ts      = Current_Timestamp(6)
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr
    WHEN NOT MATCHED THEN 
        INSERT VALUES 
            ( Site_ID        = stg.Site_ID
            , InfoKey        = stg.InfoKey
            , InfoData       = stg.InfoData
            , di_created_ts  = Current_Timestamp(6)
            , di_upd_ts      = NULL
            , di_data_src_cd = 'COA'
            , di_proc_name   = :proc_name 
            , di_batch_nbr   = :batch_nbr ) 
    ;

    delete from APP_TCA_TMP.stg_tca_dbcinfo ;
        
END;

<br><br>
# Unit Test:
run this a few times over, to emulate repeat runs

In [45]:
insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

Success: 0 rows affected

In [46]:
call APP_TCA_TBL.sp_tca_dbcinfo();

Success: 0 rows affected

In [51]:
-- should see upd_ts move, while created_ts remains static
Select * from APP_TCA_VW.coa_tca_dbcinfo
order by 1,2;

Site_ID,InfoKey,InfoData,di_created_ts,di_upd_ts,di_data_src_cd,di_proc_name,di_batch_nbr
Cust123,LANGUAGE SUPPORT MODE,Standard,2020-11-24 09:41:08.350000,2020-11-24 09:41:31.910000,COA,Unknown,0
Cust123,RELEASE,16.20.53.06,2020-11-24 09:41:08.350000,2020-11-24 09:41:31.910000,COA,Unknown,0
Cust123,VERSION,16.20.53.06,2020-11-24 09:41:08.350000,2020-11-24 09:41:31.910000,COA,Unknown,0
TDCLOUD15TD07,LANGUAGE SUPPORT MODE,Standard,2020-11-24 00:49:02.460000,2020-11-24 01:36:50.150000,COA,Unknown,0
TDCLOUD15TD07,RELEASE,16.20.53.07,2020-11-24 00:49:02.460000,2020-11-24 01:36:50.150000,COA,Unknown,0
TDCLOUD15TD07,VERSION,16.20.53.07,2020-11-24 00:49:02.460000,2020-11-24 01:36:50.150000,COA,Unknown,0
Transcend02,LANGUAGE SUPPORT MODE,Standard,2020-11-24 09:36:39.040000,2020-11-24 09:38:48.420000,COA,Unknown,0
Transcend02,RELEASE,16.20.53.06,2020-11-24 09:36:39.040000,2020-11-24 09:38:48.420000,COA,Unknown,0
Transcend02,VERSION,16.20.53.06,2020-11-24 09:36:39.040000,2020-11-24 09:38:48.420000,COA,Unknown,0
TRNAWS01,LANGUAGE SUPPORT MODE,Standard,2020-11-24 09:40:25.390000,,COA,Unknown,0


<br><br>
## Again:

In [ ]:
insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;
insert into APP_TCA_TMP.stg_tca_dbcinfo
select 
 'Transcend02' as Site_ID
, InfoKey
, InfoData
from dbc.dbcinfo as a;

In [ ]:
call APP_TCA_TMP.sp_tca_dbcinfo();

In [ ]:
-- should see upd_ts move, while created_ts remains static
Select * from APP_TCA_VW.coa_tca_dbcinfo
order by 1,2;